In [1]:
#import sys

#sys.path.append("../")
from prepare_data.utils import csv2df, clean_dataset
from prepare_data.const_predprice import  TARGET_COLS, BASE_COLS, BATCH_SIZE, DEPTH, PREDICT_LAG
from prepare_data.constants import GLASS_COLS
import numpy as np # библиотека нампи

# make train data

In [2]:
!python prepare_data\predprice_traindata.py

In [3]:
x_train_sc = np.load("./predprice/x_train_sc.npy")
x_val_sc = np.load("./predprice/x_val_sc.npy")
y_train_sc = np.load("./predprice/y_train_sc.npy")
y_val_sc = np.load("./predprice/y_val_sc.npy")

In [4]:
import tensorflow as tf           # библиотека машинного обучения
# Для генерации выборки временных рядов
TSG = tf.keras.preprocessing.sequence.TimeseriesGenerator

In [5]:
# Создаем генератор для обучения
train_datagen = TSG(x_train_sc, y_train_sc,          # В качестве параметров наши выборки
                    length = DEPTH,            # Анализируем по 21 прошедшим точкам
                    sampling_rate = 1,         # Для каждой точки
                    batch_size = BATCH_SIZE)   # Размер batch, который будем скармливать модели

# Создаем аналогичный генератор для валидации при обучении
val_datagen = TSG(x_val_sc, y_val_sc,                # В качестве параметров наши выборки
                    length = DEPTH,            # Анализируем по 21 прошедшим точкам
                    sampling_rate = 1,         # Для каждой точки
                    batch_size = BATCH_SIZE)   # Размер batch, который будем скармливать модели

print(train_datagen[0][0].shape,
      val_datagen[0][1].shape)

(64, 32, 53) (64, 5)


In [6]:
# Создадим генератор проверочной выборки, из которой потом вытащим xVal, yVal для проверки/оценки
DataGen = TSG(x_val_sc, y_val_sc,              # В качестве параметров наши выборки
              length = DEPTH,            # Анализируем по 21 прошедшим точкам
              sampling_rate = 1,         # Для каждой точки
              batch_size = len(x_val_sc))   # Размер batch будет равен длине нашей выборки


# сформируем сами массивы для проверки
XVAL, YVAL = [], []
for i in DataGen:
  XVAL.append(i[0])
  YVAL.append(i[1])

# так как это список объектов
# мы вытаскиваем нужный нам
XVAL = np.array(XVAL)[0]
YVAL = np.array(YVAL)[0]
print(XVAL.shape,YVAL.shape)

(530, 32, 53) (530, 5)


In [7]:
from construct_model.make_models import baseline

baseline_pred5price= baseline(input_shape = XVAL.shape[1:],
                    pred_num = PREDICT_LAG,
                    activ_out = "linear",
                    depth= DEPTH)


# Отображение сводки модели
#model.summary()

# дерево модели
#tf.keras.utils.plot_model(model, show_shapes=True)

In [8]:
# понижение шага
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                                 mode='min',
                                                 factor = 0.8,
                                                 patience = 3,
                                                 min_lr = 1e-9,
                                                 verbose = 1)

# остановка
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    verbose=1,
    patience=10,
    mode='min',
    restore_best_weights=True)

path_model = './model/predprice/'

# Сохранение модели
checkpoint  = tf.keras.callbacks.ModelCheckpoint(
                        filepath = path_model,
                        #save_weights_only=True,
                        monitor='val_loss',
                        mode='min',
                        save_best_only=True
                        )
# оптимизатор
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=1e-2)

# функция потерь
loss = tf.keras.losses.Huber() #.MSE #
baseline_pred5price.compile(optimizer = optimizer, loss = loss, metrics=['mae', 'mape'])

In [9]:
# обучение модели вашей
history = baseline_pred5price.fit(train_datagen,                  # генератор данных для обучения
                    validation_data = val_datagen, # генератор данных для проверки
                    epochs = 2,
                    callbacks=[early_stopping, reduce_lr, checkpoint],
                    verbose = 1)

Epoch 1/2
34/34 [==============================] - ETA: 0s - loss: 0.1460 - mae: 0.4409 - mape: 402582.4062

INFO:tensorflow:Assets written to: ./predprice\assets


INFO:tensorflow:Assets written to: ./predprice\assets


34/34 [==============================] - 434s 12s/step - loss: 0.1460 - mae: 0.4409 - mape: 402582.4062 - val_loss: 0.0159 - val_mae: 0.1537 - val_mape: 31.4995 - lr: 0.0100
Epoch 2/2
34/34 [==============================] - 17s 444ms/step - loss: 0.0414 - mae: 0.2401 - mape: 294439.3125 - val_loss: 0.0162 - val_mae: 0.1613 - val_mape: 31.4458 - lr: 0.0100
